First, TensorFlow and other libraries are imported

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
#from data import *

#===========Read all data from files contained in folder from path
def read_data(path, keyname):
    data = []
    h5name = os.listdir(path) # Read all the names of data in file (path) and store as a list

    for i in range(len(h5name)): # Repeat for every fiel in folder from path
        temp_path = path + '//' + str(h5name[i]) # Create a path for each file in the folder
        tempdata = h5py.File(temp_path, "r") # Open h5 file as "read" and save as data
        tempkeydata = tempdata[keyname] # Store data from 'kspace' key as kspacedata
        data.append(tempkeydata) # Add data from key to list
    data = np.array(data) # Turn list into numpy array
    return data

#==========Read Target Images
def read_target_images(path):
    data = read_data(path, 'reconstruction_rss') # Read dataset called reconstruction_rss in the h5 file
    # Data is a numpy array: (4,18,256,256) (files, acquisitions, images pixels)
    data_length = len(data) # store how many files have been read
    images = [] # Generate empty variable
    for i in range(data_length): # Repeat for the number of files in folder
        d = data[i] # Save d as (18,256,256)
        num_acq = d.shape[0] # Number of acquisitions in each file
        for j in range(num_acq):  # Repeat for number of acquisitions
            acquisition = d[j, ...] # Seperate image based on loop number
            images.append(acquisition) # Add image to lits of images
    images = np.array(images) # At the end of function turn list of images into numpu array
    return images

#==========Read Kspace Images and combine images from each coil fro each acquisition
def read_source_images(path):
    data = read_data(path, 'kspace') # Read dataset called reconstruction_rss in the h5 file
    # Data is a numpy array: (4, 18, 4, 256, 256) (files, acquisitions, coils, imagex, imagey)
    data_length = len(data) # store how many files have been read
    kspace_data = [] # Generate empty variable
    kspace_images = []
    for i in range(data_length): # Repeat for the number of files in folder
        d = data[i] # Save d as (18, 4, 256, 256)
        num_acq = d.shape[0] # Number of acquisitions in each file
        for j in range(num_acq):  # Repeat for number of acquisitions
            acquisition = d[j, ...] # Seperate image based on loop number
            kspace_data.append(acquisition) # Add image to lits of images
            #This is simplified to average for now-------
            coil0 = acquisition[0, ...]
            coil1 = acquisition[1, ...]
            coil2 = acquisition[2, ...]
            coil3 = acquisition[3, ...]
            kspace = (coil0+coil1+coil2+coil3)/4 # Coils added and averaged
            #--------------------------------------------
            kspace_images.append(kspace)
    kspace_data = np.array(kspace_data) # At the end of function turn list of images into numpu array
    kspace_images = np.array(kspace_images)
    return kspace_images

In [ ]:
train_images = read_source_images('//content//drive//MyDrive//DeepLearningProject//Training')
test_images = read_source_images('//content//drive//MyDrive//DeepLearningProject//Testing')
train_kspace_images = read_target_images('//content//drive//MyDrive//DeepLearningProject//Training')
test_kspace_images = read_target_images('//content//drive//MyDrive//DeepLearningProject//Testing')
print(train_kspace_images.shape) # Number of files(4)*acquisitions(18)
print(test_kspace_images.shape)  # Number of files(2)*acquisitions(18)
print(train_images.shape) # Number of files(4)*acquisitions(18)
print(test_images.shape) # Number of files(2)*acquisitions(18)
i_test = test_kspace_images # Size (36, 256, 256)
i_train = train_kspace_images # Size (36, 256, 256)
#train_kspace_images size = (36,256,256)
#test_kspace_images size = (36,256,256)
#train_images size = (36,256,256)
#test_images size = (36,256,256)


Load the dataset from the mri brain images

In [ ]:
(i_train, _), (i_test, _) = image #image is the image from gabriel's dataloding

#Normalize the data
i_train = i_train.astype('float32') / 255.
i_test = i_test.astype('float32') / 255.

print (i_train.shape)#image shape


**Autoencoder**:

Here I train a convolutional autoeconder using the function Conv2D layers. For the decoder, I use the function Conv2DTranspose layers.


**Relu** is a non-linear function that replaces any negative values in the input with zero and leaves positive values unchanged.

**Padding 'same'** means that additional rows and columns of zeros are added around the input data, so that the convolutional or pooling operation can be applied to every position.

**Strides** control the step size at which the kernel moves: a stride of 1 means the kernel moves one pixel at a time. A larger stride results in a smaller output size because the kernel covers fewer positions.

In regards to the layers.Conv2D:
layers.Conv2D(filters, kernel_size, activation, padding, strides)

Activation adam is an extension of the stochastic gradient descent optimization method.

In [ ]:
class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([ #keras is used to add layers and improve the network
      layers.Input(shape=(256, 256, 1)),#CHANGE THIS WHEN I KNOW THE SHAPE OF THE IMAGES
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=1),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=1)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()

autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(i_train, image,
                epochs=10,
                shuffle=True,
                validation_data=(i_test, image))

encoded_imgs = autoencoder.encoder(i_test).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

Display the images

In [ ]:
# Display of original images and reconstructed images
dispacq = 3  # Number of acquisitions to be displayed
numcoil = len(acq[0])  # Number of coils

for i in range(dispacq):
    plt.subplot(2, dispacq, i + 1) # Create subplot to be scaled to number of acquisitions displayed
    plt.imshow(abs(image), cmap='gray')
    plt.title('Image Domain' + str(i), fontdict=font)
    plt.subplot(2, dispacq, (i + dispacq+1))
    plt.imshow(abs(decoded_imgs), cmap='gray')
    plt.title('Reconstructed images' + str(i), fontdict=font)
plt.tight_layout()
plt.show()
